In [0]:
# Import standard libraries
import numpy as np
import pandas as pd
# Import CCDC libraries
import ccdc
from ccdc import search, io, molecule
from ccdc.search import SimilaritySearch, SubstructureSearch, MoleculeSubstructure
from ccdc.io import MoleculeReader, CrystalReader, EntryReader
import csv
# Import RDKit features
import rdkit.Chem
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole

In [0]:
# Define a function such that the duplicate CCDC Identifiers will be removed
def Remove(duplicate):
    seen= set()
    final_list = []
    for num in duplicate: 
        if num not in seen:
            
            seen.add(num)
            final_list.append(num)    
    return final_list 
      

In [0]:
# Define the similarity search function, with Tanimoto 35%
# Save the resulted stuctures on a list after removing duplicate CCDC Identifiers

sim_list = []
def SimSearch(mol):    
    csd_reader = MoleculeReader('CSD')
    multi_mol = np.empty([])   
    similarity_query = ccdc.search.SimilaritySearch(mol, threshold= 0.35)
    similarity_query.settings.only_organic = True
    similarity_query.settings.not_polymeric = True
    similarity_query.settings.no_errors = True
    similarity_query.settings.no_ions = True
    similarity_query.settings.has_3d_coordinates = True
    similarity_query.settings.no_metals = True
    sim_hits = similarity_query.search()
    #print (len(sim_hits))
    for h in sim_hits:
        if str(csd_reader.molecule(h.identifier).smiles) != 'None':
            sim_list.append(h.identifier)
    return Remove(sim_list)

In [0]:
# Get an array with the common solvents smiles and remove the organic solvents form that list
# Keep only benzene-derived solvent as they might hold
# important information about the type of cocrystals we investigate

df = pd.read_csv("solvent_smiles.txt", header=None)
lista = df.iloc[:, 0].values

organic_solvents = ['c1ccccc1', 'Clc1ccccc1', 'Cc1ccccc1', 'C1CCOC1' ,'c1ccncc1', 'Cc1ccc(C)cc1', 'Cc1ccccc1C']

solvents = [x for x in lista if x not in organic_solvents]

In [0]:
# Read the initial molecules
filepath = "initial.txt"
mol_reader = io.MoleculeReader(filepath, format='identifiers')

for mol in mol_reader:
    print(mol.identifier)

FLUANT
CORANN01
DNAPAN
CORONE
ZZZOYC01
PENCEN
TRIPHE
PHENAN


In [0]:
csd_reader = MoleculeReader('CSD')
for mol_ in mol_reader:
    mol = csd_reader.molecule(mol_.identifier)
    sim_list=SimSearch(mol)
print(len(sim_list))

8113


In [0]:
# Find the INCHI numbers of the CSD entries and filter the enties according to their INCHI number to make sure that

inchi =[]
def inchi_filter(lista):
    for mol1 in lista:
        mol = csd_reader.molecule(mol1)
        #mol.add_hydrogens('missing')
        mol.assign_bond_types()
        #Chem.SanitizeMol(mol) 
        mol_block =mol.to_string('sdf')
    
        rdkit_mol = rdkit.Chem.MolFromMolBlock(mol_block, sanitize=False)
        try:
            Chem.SanitizeMol(rdkit_mol)
            inchi_str = rdkit.Chem.inchi.MolToInchi(rdkit_mol)
    
        except ValueError as e:
            sm = str(e)
        inchi.append(inchi_str)
    return Remove(inchi)       
# Remove duplicate INCHI numbers
rm_inchi=inchi_filter(sim_list)

In [0]:
# Define a dictionary where each CSD entry will be correlated with its INCHI number such that we can get again the ccdc
# identifiers after the filtration

keys = inchi
values = sim_list

d = {key:value for key, value in zip(keys, values)}

# List all the unique entries

all_unique = []
for num in rm_inchi:
    all_unique.append(d[num])
print (len(all_unique))

6657


In [0]:
# Define a dictionary where all the CSD identifiers are assigned their smiles

lista =[]
for mol1 in all_unique:
    mol = csd_reader.molecule(mol1)
    lista.append(mol.smiles)
    
    
keys = lista
values = all_unique

dict = {key:value for key, value in zip(keys, values)}


In [0]:
# construct three lists according to the number of components of the crystals after the similarity search
# Start with constructing the list of single molecule crystals

one_component = []

for mol1 in all_unique:
    mol = csd_reader.molecule(mol1)
    
    if "." not in str(csd_reader.molecule(mol.identifier).smiles):
        one_component.append(mol.identifier)                           
    

In [0]:
#Print the number of single molecule crystals
len(one_component)

2681

The molecules found the the one_component list, that represents the single molecule crystals are further processed to a substracture search. Setting each single molecule as one component all the structures that contain that molecule with be detected.

In [0]:
# Define the substructure search function using the same parameters as for the similarity search
substructure_list = []
def OnlyCoCryst(mol):
    csd_reader = MoleculeReader('CSD')

    mol_substructure = MoleculeSubstructure(mol.components[0])
    substructure_search = SubstructureSearch()
    substructure_search.settings.only_organic = True
    substructure_search.settings.has_3d_coordinates =True
    substructure_search.settings.no_metals = True
    substructure_search.settings.no_ions = True
    substructure_search.settings.no_errors = True
    substructure_search.settings.not_polymeric = True        
    sub_id = substructure_search.add_substructure(mol_substructure)
    hits = substructure_search.search()
    #print len(hits)
    for h in hits:
        if "."  in str(csd_reader.molecule(h.identifier).smiles):
            substructure_list.append(h.identifier)        
    return Remove(substructure_list)

In [0]:
one_component=[x.encode('utf-8') for x in one_component]
len(one_component)
for mol1 in one_component:
    mol = csd_reader.molecule(mol1)
    finale_single = OnlyCoCryst(mol)
        

In [0]:
# Add the co-crystals from the substructure search in the list of co-crystals if they do not already exist there
x=[x for x in finale_single if x not in all_unique]
all_unique=all_unique+x

In [0]:
len(all_unique)

In [0]:
# Split the co-crystals list into two sublists including those with two molecules and those with more than two
two_components = []
threeplus_components = []

for mol1 in all_unique:
    mol = csd_reader.molecule(mol1)

    if "."  in str(csd_reader.molecule(mol.identifier).smiles):
        if len(csd_reader.molecule(mol.identifier).components) == 2:
            two_components.append(mol.identifier)
                    
        else:
            threeplus_components.append(mol.identifier)                               
    

First deal with the crystals found with three and more components (threeplus_components list). After removing the common solvents, we keep only the structures with two different molecules and could their ratio and Z'.

In [0]:
# Define a function that cleans the list from solvents

final_three =[]
for mol1 in threeplus_components:
    mol = csd_reader.molecule(mol1)
    #print(len(mol.components))
    
    for i in range(0, (len(mol.components))):
        if mol.components[i].smiles not in solvents:
            final_three.append(mol.identifier)
            


In [0]:
# Keep only the crystals that have two diffent molecules in various ratios
finale=[]
for mol in final_three:
    if final_three.count(mol) == len(csd_reader.molecule(mol).components):
        finale.append(mol)
fin_three=Remove(finale) 

In [0]:
print(len(threeplus_components))
print (len(final_three))
print (len(fin_three))


In [0]:
# Count Z and the ratio of each molecule in the co-crystals
Z =[]
smiles3 =[]
component1, component2 =[], []
fin=[]
for mol1 in fin_three:
    mol = csd_reader.molecule(mol1)
    #Z.append(len(mol.components))
    smi= mol.smiles
    smi = smi.split('.')
    if len(Remove(smi)) ==2:  # We make sure that the structure consist of two different molecules
        fin.append(mol.identifier)
        Z.append(len(mol.components))
        component1.append(Remove(smi)[0])
        component2.append(Remove(smi)[1])
        smiles3.append(smi.count(mol.components[0].smiles))

In [0]:
# Construct the table with the multi-component crystals
df1 = pd.DataFrame(component1, columns=['smiles1'])
df4 = pd.DataFrame(component2, columns=['smiles2'])
df2 = pd.DataFrame(Z, columns=['Z'])
df3 = pd.DataFrame(smiles3, columns=['ratio1'])
df5 = pd.DataFrame(fin,columns=['Identifier'])
df6 = pd.DataFrame(df2["Z"]-df3["ratio1"], columns=['ratio2'])
dataset3 = pd.concat([df5, df1, df4, df2, df3, df6], axis=1)
dataset3.tail()

Deal with the two_components list, where all the crystals with two components in molecular ratios 1:1 and Z=2 exist. First remove the common solvents from the structures

In [0]:
# Define a function that cleans the list from solvents

final_two =[]
for mol1 in two_components:
    mol = csd_reader.molecule(mol1)
    #print(len(mol.components))
    
    for i in range(0, (len(mol.components))):
        if mol.components[i].smiles not in solvents:
            final_two.append(mol.identifier)
            


# Keep only the crystals that have two diffent molecules in various ratios
finale2=[]
for mol in final_two:
    if final_two.count(mol) == len(csd_reader.molecule(mol).components):
        finale2.append(mol)
final_two=Remove(finale2) 

# Final list with two-component-co-crystals with Z=2 and ratios 1:1
print(len(two_components))
print (len(final_two))

In [0]:
# Count Z and the ratio of each molecule in the co-crystals
Z2 =[]
smiles_2 =[]
component_1, component_2 =[], []
fin2=[]
for mol1 in final_two:
    mol = csd_reader.molecule(mol1)
    
    smi= mol.smiles
    smi = smi.split('.')
    if len(Remove(smi))==2:  # We make sure that the structure consist of two different molecules
        fin2.append(mol.identifier)
        Z2.append(len(mol.components))
        component_1.append(Remove(smi)[0])
        component_2.append(Remove(smi)[1])
        smiles_2.append(smi.count(mol.components[0].smiles))

In [0]:
# Construct the table with the multi-component crystals
df_1 = pd.DataFrame(component_1, columns=['smiles1'])
df_4 = pd.DataFrame(component_2, columns=['smiles2'])
df_2 = pd.DataFrame(Z2, columns=['Z'])
df_3 = pd.DataFrame(smiles_2, columns=['ratio1'])
df_5 = pd.DataFrame(fin2,columns=['Identifier'])
df_6 = pd.DataFrame(df_2["Z"]-df_3["ratio1"], columns=['ratio2'])
dataset2 = pd.concat([df_5, df_1, df_4, df_2, df_3, df_6], axis=1)

In [0]:
dataset=pd.concat([dataset2, dataset3], ignore_index=True)
dataset.shape

In [0]:
# Save the extracted co-crystals in a csv file to be further processed via Pipeline Pilot to remove acidic hydrogens
pd.DataFrame(dataset).to_csv('final_cocrystals.csv')